# CADS API: constraints tests

In [69]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
import os
import xarray as xr

import cads_api_client

In [71]:
api_url = os.getenv("CADS_API_URL", "http://cds2-dev.bopen.eu/api")
api_url

'http://cds2-dev.bopen.eu/api'

## Client instantiation

In [72]:
client = cads_api_client.ApiClient(api_url, api_key="mysecretpat")
client

ApiClient(url='http://cds2-dev.bopen.eu/api', api_key='mysecretpat')

## Check Collections ids

In [73]:
client.collections().collection_ids()

['cams-global-reanalysis-eac4-monthly',
 'reanalysis-era5-single-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-land-monthly-means',
 'derived-near-surface-meteorological-variables']

## Constraints API validation

**Objective**: verify the correct functioning of the constraints API.

**Expected result:** client.valid_values() returns values that can be downloaded together with the input.

Consider the `"reanalysis-era5-single-levels"` dataset 

In order to check all available values for a dataset leave the `request` param empty  

In [92]:
client.valid_values("reanalysis-era5-single-levels", {})


{'product_type': ['reanalysis',
  'ensemble_members',
  'ensemble_spread',
  'ensemble_mean'],
 'variable': ['mean_vertically_integrated_moisture_divergence',
  'trapping_layer_base_height',
  'forecast_albedo',
  'high_vegetation_cover',
  'ice_temperature_layer_1',
  'mean_surface_net_short_wave_radiation_flux',
  'surface_runoff',
  'uv_visible_albedo_for_direct_radiation',
  'convective_inhibition',
  'vertical_integral_of_potential_internal_and_latent_energy',
  'wave_spectral_directional_width',
  'lake_ice_depth',
  'mean_top_net_long_wave_radiation_flux_clear_sky',
  'vertical_integral_of_kinetic_energy',
  'mean_surface_downward_long_wave_radiation_flux',
  'lake_mix_layer_temperature',
  'snowfall',
  'mean_direction_of_total_swell',
  'mean_top_downward_short_wave_radiation_flux',
  'maximum_individual_wave_height',
  'free_convective_velocity_over_the_oceans',
  'significant_wave_height_of_second_swell_partition',
  'normalized_stress_into_ocean',
  'mean_sub_surface_runoff

### Test 1 - Data availability

#### 1.1 - Availability limits
The `"reanalysis-era5-single-levels"` dataset stops the 31th of october. Checking which values are valid for the year `2022` should return months up to 10 (october).


In [74]:
available_months = client.valid_values("reanalysis-era5-single-levels", {"year" : "2022"})["month"]
available_months.sort()
available_months

['01', '02', '03', '04', '05', '06', '07', '08']

#### 1.2  - Unavailable data

For a valid key, if the requested value is not valid for the current dataset, the API will suggest a list of valid values:


In [109]:
client.valid_values("reanalysis-era5-single-levels", {"year":"-1000"})

{'product_type': [],
 'variable': [],
 'year': ['1970',
  '1993',
  '1997',
  '2006',
  '1965',
  '1962',
  '1979',
  '1964',
  '2013',
  '2008',
  '1989',
  '1966',
  '2015',
  '2012',
  '2011',
  '2001',
  '1990',
  '1981',
  '1977',
  '2020',
  '1991',
  '1980',
  '2009',
  '1978',
  '2003',
  '2017',
  '1982',
  '2019',
  '2002',
  '1987',
  '1994',
  '1959',
  '1986',
  '1996',
  '2000',
  '1971',
  '1995',
  '1973',
  '2021',
  '1975',
  '1984',
  '1961',
  '1972',
  '2010',
  '1968',
  '2007',
  '1969',
  '2005',
  '2018',
  '1992',
  '1967',
  '1985',
  '1974',
  '1960',
  '1988',
  '1963',
  '2016',
  '1999',
  '1998',
  '2004',
  '1983',
  '2022',
  '1976',
  '2014'],
 'month': [],
 'day': [],
 'time': [],
 'format': ['grib', 'netcdf']}

### Test 2 - Restricted params

Consider the `"cams-global-reanalysis-eac4-monthly"` dataset. `"product_type":"monthly_mean"` should not be compatible with any `"time"` value.

In [95]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {"product_type": "monthly_mean"})["time"]

[]

On the countrary, if `"product_type"` is `"monthly_mean_by_hour_of_day"`, all `"time"` values should be available

In [94]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {'product_type': 'monthly_mean_by_hour_of_day'})["time"]

['00:00', '03:00', '18:00', '09:00', '15:00', '21:00', '06:00', '12:00']

the reverse is also true:

In [97]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {"time": ['00:00', '03:00', '18:00', '09:00', '15:00', '21:00', '06:00', '12:00']})["product_type"]

['monthly_mean_by_hour_of_day']

### Test 3 - invalid combination
Consider the `"cams-global-reanalysis-eac4-monthly"` dataset. There should be no valid combinations for `"time":"12:00"` and `"product type":"montly_mean"`.


In [78]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {"product_type" : "montly_mean", "time":"12:00"})

{'variable': [],
 'pressure_level': [],
 'model_level': [],
 'year': [],
 'month': [],
 'product_type': ['monthly_mean_by_hour_of_day'],
 'time': [],
 'format': ['grib', 'netcdf']}

The API responce suggests that adding `"monthly_mean_by_hour_of_day"` to the request may produce some valid values. 

Indeed:

In [114]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {"product_type" : ["montly_mean", "monthly_mean_by_hour_of_day"] , "time":"12:00"})

{'variable': ['vertically_integrated_mass_of_dust_aerosol_0.55-9um',
  'sulphur_dioxide',
  'ice_temperature_layer_1',
  'nitrogen_dioxide',
  'leaf_area_index_low_vegetation',
  'hydrophobic_organic_matter_aerosol_mixing_ratio',
  'total_column_methane',
  'particulate_matter_2.5um',
  'total_column_nitrogen_monoxide',
  'vertically_integrated_mass_of_dust_aerosol_9-20um',
  'relative_humidity',
  'total_column_propane',
  'vertically_integrated_mass_of_sea_salt_aerosol_5-20um',
  'vertically_integrated_mass_of_sulphur_dioxide',
  'ozone',
  '2m_temperature',
  'dust_aerosol_optical_depth_550nm',
  'formaldehyde',
  'vertically_integrated_mass_of_dust_aerosol_0.03-0.55um',
  'sulphate_aerosol_mixing_ratio',
  'total_column_hydroxyl_radical',
  'snow_albedo',
  'vertically_integrated_mass_of_hydrophilic_organic_matter_aerosol',
  'specific_humidity',
  'sea_salt_aerosol_0.03-0.5um_mixing_ratio',
  'total_column_nitrogen_dioxide',
  'sea_salt_aerosol_5-20um_mixing_ratio',
  'potential_v

### Test 4 - Always valid params

Some params are always valid, meaning they can go along with any other combination. These are always returned by the API:

In [119]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {"product_type" : "X", "variable":"X"})

{'variable': [],
 'pressure_level': [],
 'model_level': [],
 'year': [],
 'month': [],
 'product_type': [],
 'time': [],
 'format': ['grib', 'netcdf']}

### Test 5 - Invalid Keys


If some param with invalid or misspelled name is requested the API throws an error 

In [120]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {"invalid_param":"X"})

{'variable': [],
 'pressure_level': [],
 'model_level': [],
 'year': [],
 'month': [],
 'product_type': [],
 'time': [],
 'format': ['grib', 'netcdf']}